a) Import all the necessary libraries

In [2]:
# import modules
%pylab inline
import os 
import numpy as np
import pandas as pd
from scipy.misc import imread
from sklearn.metrics import accuracy_score

Populating the interactive namespace from numpy and matplotlib


ImportError: cannot import name 'imread' from 'scipy.misc' (c:\users\suraj\appdata\local\programs\python\python37\lib\site-packages\scipy\misc\__init__.py)

 Let’s set a seed value, so that we can control our models randomness

In [3]:
# To stop potential randomness
seed = 128
rng = np.random.RandomState(seed)

c) The first step is to set directory paths, for safekeeping!

In [4]:
root_dir = os.path.abspath('.')
data_dir = os.path.join(root_dir, 'data') 
# check for existence
os.path.exists(root_dir), os.path.exists(data_dir)

(True, False)

##### Data Loading and Preprocessing

a) Now let us read our datasets. These are in .csv formats and have a filename along with the appropriate labels

In [ ]:
# load dataset
train = pd.read_csv(os.path.join(data_dir, 'Train', 'train.csv'))
test = pd.read_csv(os.path.join(data_dir, 'Test.csv')) 

sample_submission = pd.read_csv(os.path.join(data_dir, 'Sample_Submission.csv')) 

train.head()

In [ ]:
# print an image
img_name = rng.choice(train.filename)
filepath = os.path.join(data_dir, 'Train', 'Images', 'train', img_name) 

img = imread(filepath, flatten=True) 

pylab.imshow(img, cmap='gray')
pylab.axis('off')
pylab.show()

For easier data manipulation, let’s store all our images as numpy arrays

In [ ]:
# load images to create train and test set
temp = []
for img_name in train.filename:
    image_path = os.path.join(data_dir, 'Train', 'Images', 'train', img_name)
    img = imread(image_path, flatten=True)
    img = img.astype('float32')
    temp.append(img

train_x = np.stack(temp) 
train_x /= 255.0
train_x = train_x.reshape(-1, 784).astype('float32') 

temp = []
for img_name in test.filename:
    image_path = os.path.join(data_dir, 'Train', 'Images', 'test', img_name)
    img = imread(image_path, flatten=True)
    img = img.astype('float32')
    temp.append(img) test_x = np.stack(temp) 

test_x /= 255.0
test_x = test_x.reshape(-1, 784).astype('float32') 

train_y = train.label.values

As this is a typical ML problem, to test the proper functioning of our model we create a validation set. Let’s take a split size of 70:30 for train set vs validation set

In [ ]:
# create validation set
split_size = int(train_x.shape[0]*0.7)

train_x, val_x = train_x[:split_size], train_x[split_size:]
train_y, val_y = train_y[:split_size], train_y[split_size:]

 Model Building
a) Now comes the main part! Let us define our neural network architecture. We define a neural network with 3 layers  input, hidden and output. The number of neurons in input and output are fixed, as the input is our 28 x 28 image and the output is a 10 x 1 vector representing the class. We take 50 neurons in the hidden layer. 

In [ ]:
import torch from torch.autograd import Variable
# number of neurons in each layer
input_num_units = 28*28

hidden_num_units = 500

output_num_units = 10

# set remaining variables
epochs = 5
batch_size = 128
learning_rate = 0.001

In [5]:
# define model
model = torch.nn.Sequential(
    torch.nn.Linear(input_num_units, hidden_num_units),
    torch.nn.ReLU(),
    torch.nn.Linear(hidden_num_units, output_num_units),
)
loss_fn = torch.nn.CrossEntropyLoss() 
# define optimization algorithm
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
## helper functions

# preprocess a batch of dataset
def preproc(unclean_batch_x):
   """Convert values to range 0-1"""
   temp_batch = unclean_batch_x / unclean_batch_x.max()
    
   return temp_batch 

# create a batch
def batch_creator(batch_size):  
    dataset_name = 'train'
    dataset_length = train_x.shape[0]

    batch_mask = rng.choice(dataset_length, batch_size)
    batch_x = eval(dataset_name + '_x')[batch_mask]

    batch_x = preproc(batch_x)

    if dataset_name == 'train':
       batch_y = eval(dataset_name).ix[batch_mask, 'label'].values
    
    return batch_x, batch_y
# train network
total_batch = int(train.shape[0]/batch_size)

for epoch in range(epochs):
    avg_cost = 0
    for i in range(total_batch):
       # create batch
       batch_x, batch_y = batch_creator(batch_size)
       
       # pass that batch for training
       x, y = Variable(torch.from_numpy(batch_x)), Variable(torch.from_numpy(batch_y), requires_grad=False)
       pred = model(x)
      
       # get loss
       loss = loss_fn(pred, y)
       
       # perform backpropagation
       loss.backward()
       optimizer.step()

       avg_cost += loss.data[0]/total_batch
    print(epoch, avg_cost)
# get training accuracy
x, y = Variable(torch.from_numpy(preproc(train_x))), Variable(torch.from_numpy(train_y), requires_grad=False)

pred = model(x)
final_pred = np.argmax(pred.data.numpy(), axis=1) 

accuracy_score(train_y, final_pred)
# get validation accuracy
x, y = Variable(torch.from_numpy(preproc(val_x))), Variable(torch.from_numpy(val_y), requires_grad=False)

pred = model(x)
final_pred = np.argmax(pred.data.numpy(), axis=1) 

accuracy_score(val_y, final_pred)

NameError: name 'torch' is not defined